# Lab 2 Week 3
## Exercise 5a
- f<sub>T</sub> = 10f<sub>u</sub> > 1GHz
- V<sub>SD2</sub> = V<sub>SG2</sub>
- maximize A<sub>v</sub>

- Algorithm
  - Variables: W_p, W_n, L_p, L_n, Vgs_n, Vb(Vsg_p), Vout(Vds_n or Vsd_p)
  - Constrains: Vsd_p = Vsg_p, fT_n = fT_p = 1GHz, fu = 0.1GHz
  - -> 7 - 4 = 3 variables
  - We can choose 3 independent variables: Vout, L_p, L_n

In [1]:
# Always run this section first
import numpy as np
import matplotlib.pyplot as plt
from look_up import *
nch = importdata("nch_1v.mat")
pch = importdata("pch_1v.mat")

- try some functions for PMOS, since V<sub>DS</sub> and V<sub>GS</sub> are negative

In [6]:
print(pch["VDS"])
print(look_up_basic(pch, "GM", vds = 0.55, l = 0.18, vsb = 0, vgs = 0.55))

[0.    0.025 0.05  0.075 0.1   0.125 0.15  0.175 0.2   0.225 0.25  0.275
 0.3   0.325 0.35  0.375 0.4   0.425 0.45  0.475 0.5   0.525 0.55  0.575
 0.6   0.625 0.65  0.675 0.7   0.725 0.75  0.775 0.8   0.825 0.85  0.875
 0.9   0.925 0.95  0.975 1.    1.025 1.05  1.075 1.1  ]
0.00018873166171191972


- In PMOS LUT, use V<sub>SD</sub>, V<sub>SG</sub> and other positive values!!!
- The defined funcitons in look_up.py use V<sub>DS</sub> as V<sub>SD</sub> for PMOS

In [ ]:
# Set the parameters
fu = 1e8
fT = fu*10
CL = 1e-12
VDD = 1.1
Vout_range = np.arange(0.3, 0.7, 0.05) # [0.3, 0.35... 0.7]
L_range = nch["L"]
gm_ID_range = np.arange(5, 30, 1) # [5, 6... 29]

# Define the maxtrices under different Vout and L_n, the maximzied parameter under different gm_ID_n
best_L_p_matrix = np.zeros((len(Vout_range), len(L_range)))
best_Av_matrix = np.zeros((len(Vout_range), len(L_range)))
best_gm_ID_n_matrix = np.zeros((len(Vout_range), len(L_range)))

# Sweep over Vout
for i in range(len(Vout_range)):
    Vout = Vout_range[i]
    VSD_p = VDD - Vout
    VSG_p = VSD_p
    # Find the best length of PMOS
    gds_ID_p_vector = look_up_basic(pch, "ID", vds = VSD_p, l = L_range, vsb = 0, vgs = VSG_p) # gds/id under different L_p
    fT_p_vector = look_up_basic(pch, "GM_CGG", vds = VSD_p, l = L_range, vsb = 0, vgs = VSG_p)/(2*np.pi) # fT under different L_p
    
    # Find satisfied(maximum) L_p with fT_p > 1GHz, since fT_p is decreasing with L_p increasing
    for j in range(len(fT_p_vector)):
        if fT_p_vector[j] > fT:
            L_p_optimized = L_range[j]
            gds_ID_p_optimized = gds_ID_p_vector[j] # find the last L_p(maximum) that satisfies fT_p > 1GHz
    
    is_empty_
    # Find the best Av under the optimized L_p
    for j in range(len(L_range)):
        L_n = L_range[j]
        VDS_n = Vout
        gds_ID_n_vector = look_up_vs_gm_id(nch, "GDS_ID", gm_id=gm_ID_range, vds = VDS_n, l = L_n, vsb = 0) # gds/id under different gm_ID_n
        fT_n_vector = look_up_vs_gm_id(nch, "GM_CGG", gm_id=gm_ID_range, vds = VDS_n, l = L_n, vsb = 0)/(2*np.pi) # fT under different gm_ID_n
        Av_vector = gm_ID_range / (gds_ID_n_vector + gds_ID_p_optimized) # is legal in nparray because gds_ID_p_optimized will be broadcasted into a vector
        
        # Find gm_ID_n that satisfies fT_n > 1GHz that maximizes Av
        not_empty = 0
        for k in range(len(fT_n_vector)):
            if fT_n_vector[k] > fT:
                if Av_vector[k] > best_Av_matrix[i, j]:
                    best_Av_matrix[i, j] = Av_vector[k]
                    best_gm_ID_n_matrix[i, j] = gm_ID_range[k]
                    not_empty = 1
        
        if (not_empty == 0):
            best_Av_matrix[i, j] = float('NaN')
            best_gm_ID_n_matrix[i, j] = float('NaN')
        

                
                
                
    

